# Working with LES data: dry boundary layer

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

the_file = Path.home() / "Dropbox/phil_files/a500/data" / "case_60_10.nc"
if not the_file.exists():
    raise ValueError(f"{the_file} not found")

## Find the netcdf file an dump its contents

In [ ]:
the_file

In [ ]:
!ncdump -h /Users/phil/Dropbox/phil_files/a500/data/case_60_10.nc

Netcdf file layout:  10 groups corresponding to 10 different ensemble members.  Small slice of larger domain of LES run with surface heat flux of 60 W/m^2 and stable layer with dT/dz = 10 K/km.  Snapshots every 10 minutes for 8 hours.

## Open the first ensemble member c1

Look at the last 5 values of z, pressure

From the ncdump we know that there are global variables and attributes as
well as the 10 ensemble members.  So get two xarray dataset pointers:

- ds1 for the first ensemble member
- the_ds for the global variables

In [ ]:
ds1 = xr.open_dataset(the_file,group="c1")
the_ds = xr.open_dataset(the_file)

In [ ]:
the_ds['z'][-5:]

In [ ]:
ds1

In [ ]:
temp = ds1['TABS']
wvel= ds1['W']
z=the_ds['z']
press=the_ds['press']

## dump height and pressure

In [ ]:
(z[-5:].data,press[-5:].data)

## Function to convert to $\theta$

In [ ]:
def make_theta(temp,press):
    """
      temp in K
      press in Pa
      returns theta in K
    """
    p0=1.e5
    Rd=287.
    cpd=1004.
    theta=temp*(p0/press)**(Rd/cpd)
    
    return theta
    

## Get the horizontal average

In [ ]:
temp_mean=temp.mean(dim=["x","y"])

In [ ]:
temp_mean[0,-5:]

In [ ]:
## Plot the timestep theta,height profiles for c1

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(8,8))
ntimes, nheights = temp_mean.shape
for i in np.arange(0,ntimes,3):
    theta = make_theta(temp_mean[i,:],press)
    ax.plot(theta,z)
ax.set(xlabel=r'$\overline{\theta}$ (K)',ylabel='height (m)',
       title='LES dry run:  surface flux=60 $W\,m^{-2}$, $\Gamma$=10 K/km');

    

## Assignment -- targeting Tuesday

- Write functions that find the ensemble mean $\langle \zeta \rangle$ and the perturbation about that mean $\zeta^\prime$ an arbitrary variable $\zeta$
- Use those functions to calculate the horizontally averaged vertical entropy flux $\overline{w^\prime\,\theta^\prime}$ as a function of height and make a plot similar to the figure above